In [1]:

from enhancing.modules.stage1.vitvqgan import ViTVQ
from enhancing.modules.stage1.layers import ViTEncoder as Encoder, ViTDecoder as Decoder
from enhancing.modules.stage1.quantizers import VectorQuantizer, GumbelQuantizer
from enhancing.utils.general import get_config_from_file, initialize_from_config

import torch

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.6.2) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [ ]:

config = get_config_from_file("/home/nvidia/semcom_ws/src/commun/commun/imagenet_vitvq_encoder_small.yaml")
vitvq = initialize_from_config(config.model)
for param in vitvq.parameters():
    param.requires_grad = False

for b in [1, 2, 3, 4, 6]:
    
    # torch.onnx.export(
    #     model.encoder, dummy_input,
    #     f"vqgan_encoder_{b}.onnx",
    #     export_params=True,
    #     opset_version=13,
    #     dynamic_axes=None,          # IMPORTANT for INT8
    #     do_constant_folding=True
    # )
    dummy_input = torch.randn(b, 3, 256, 256, device="cuda")
    torch.onnx.export(
        vitvq, dummy_input, f"./encoder_small_{b}.onnx",
        export_params=True,
        opset_version=13,
        dynamic_axes=None,          # IMPORTANT for INT8
        do_constant_folding=True
    )
    print(f"ONNX model saved: encoder_small_{b}.onnx")
    del dummy_input

# trtexec --onnx=encoder_small_1.onnx --fp16

Restored from /home/nvidia/semcom_ws/src/commun/commun/checkpoint/imagenet_vitvq_small.ckpt
ONNX model saved: encoder_small_1.onnx
ONNX model saved: encoder_small_2.onnx
ONNX model saved: encoder_small_3.onnx
ONNX model saved: encoder_small_4.onnx
ONNX model saved: encoder_small_6.onnx


In [ ]:

config = get_config_from_file("/home/nvidia/semcom_ws/src/commun/commun/imagenet_vitvq_base.yaml")
vitvq: ViTVQ = initialize_from_config(config.model)
vitvq.init_from_ckpt("/home/nvidia/semcom_ws/src/commun/commun/checkpoint/imagenet_vitvq_base.ckpt")
for param in vitvq.parameters():
    param.requires_grad = False
vitvq.eval()
# vitvq.cuda()
print()

In [ ]:
encoder = vitvq.encoder

dummy_input = torch.randn(1, 3, 256, 256)
torch.onnx.export(vitvq.encoder, dummy_input, "./encoder_base.onnx", export_params=True)
print("ONNX model saved: encoder_base.onnx")

In [ ]:
encoder = vitvq.encoder

dummy_input = torch.randn(1, 1024, 512)
torch.onnx.export(vitvq.decoder, dummy_input, "./decoder_base.onnx", export_params=True)
print("ONNX model saved: decoder_base.onnx")

In [ ]:
# trtexec --onnx=encoder.onnx --saveEngine=encoder.trt --fp16

In [ ]:

config = get_config_from_file("/home/nvidia/semcom_ws/src/commun/commun/imagenet_vitvq_base.yaml")
encoder = Encoder(image_size=256, patch_size=8, dim=768, depth=12, heads=12, mlp_dim=3072)
dummy_input = torch.randn(1, 3, 256, 256)
torch.onnx.export(encoder, dummy_input, "./encoder_base.onnx", export_params=True)
# vitvq.cuda()
print()